In [1]:
import pandas as pd
import numpy as np
import sys, os

from scipy.stats import norm

### Data

In [2]:
df = pd.read_csv("data/filtered_tsla_options.csv")

### Netwon's Method

In [222]:
def blackscholes(S, K, T, v, r, q, t):
    
    d1 = np.log(S / K) + (r - q + v * v * 0.5) * T
    d1 /= v * np.sqrt(T)
    d2 = d1 - v * np.sqrt(T)
    
    price = t * S * np.exp(-q * T) * norm.cdf(t * d1)
    price -= t * K * np.exp(-r * T) * norm.cdf(t * d2)
    
    return price

def vega(S, K, T, v, r, q, t):
    
    d1 = np.log(S / K) + (r - q + v * v * 0.5) * T
    d1 /= v * np.sqrt(T)
    
    return S * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)

In [318]:
M = ((df.bid_price + df.ask_price) * 0.5).values
L = df.option_price.values
S = df.stock_price.values
K = df.strike_price.values
T = df.days_to_expiry.values / 365
v = df.implied_volatility.values / 100
r = df.rate.values / 100
q = df.dividend_yield.values / 100
t = df.option_type.map({"C" : 1, "P" : -1})

### Solve

In [320]:
v = pd.Series([0.5]*len(df))

In [335]:
e = (blackscholes(S, K, T, v, r, q, t) - L) / vega(S, K, T, v, r, q, t)
v = v - e
v, e

(0        1.222788
 1        2.678086
 2        1.478001
 3        1.548642
 4        1.185568
            ...   
 10269    0.681245
 10270    0.697113
 10271    0.654956
 10272    0.674039
 10273    0.676988
 Length: 10274, dtype: float64,
 0       -5.170774e-16
 1        0.000000e+00
 2        4.765366e-16
 3       -9.343153e-16
 4        9.749878e-16
              ...     
 10269    9.694787e-17
 10270    0.000000e+00
 10271    0.000000e+00
 10272    0.000000e+00
 10273    1.758935e-16
 Name: option_type, Length: 10274, dtype: float64)